# 🚀 GPT

This is a revised notebook by H Qin, October 23, 2025

In this notebook, we'll walk through the steps required to train your own GPT model on the wine review dataset

The code is adapted from the excellent [GPT tutorial](https://keras.io/examples/generative/text_generation_with_miniature_gpt/) created by Apoorv Nandan available on the Keras website.

In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import json
import re
import string
from IPython.display import display, HTML

import tensorflow as tf
from tensorflow.keras import layers, models, losses, callbacks

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

## 0. Parameters <a name="parameters"></a>

In [2]:
VOCAB_SIZE = 10000
MAX_LEN = 80
EMBEDDING_DIM = 256
KEY_DIM = 256
N_HEADS = 2
FEED_FORWARD_DIM = 256
VALIDATION_SPLIT = 0.2
SEED = 42
LOAD_MODEL = False
BATCH_SIZE = 32
EPOCHS = 5

## 1. Load the data <a name="load"></a>

In [6]:
import pandas as pd

# Load the full dataset
df = pd.read_csv("/content/winemag-data_first150k.csv")
wine_data = df.to_dict('records')

In [9]:
import csv

with open("/content/winemag-data_first150k.csv", newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)  # Automatically treats the first line as header
    wine_data = list(reader)

In [10]:
wine_data[10]

{'': '10',
 'country': 'Italy',
 'description': "Elegance, complexity and structure come together in this drop-dead gorgeous winethat ranks among Italy's greatest whites. It opens with sublime yellow spring flower, aromatic herb and orchard fruit scents. The creamy, delicious palate seamlessly combines juicy white peach, ripe pear and citrus flavors while white almond and savory mineral notes grace the lingering finish.",
 'designation': 'Ronco della Chiesa',
 'points': '95',
 'price': '80.0',
 'province': 'Northeastern Italy',
 'region_1': 'Collio',
 'region_2': '',
 'variety': 'Friulano',
 'winery': 'Borgo del Tiglio'}

In [11]:
# Filter the dataset
filtered_data = [
    "wine review : "
    + x["country"]
    + " : "
    + x["province"]
    + " : "
    + x["variety"]
    + " : "
    + x["description"]
    for x in wine_data
    if x["country"] is not None
    and x["province"] is not None
    and x["variety"] is not None
    and x["description"] is not None
]

In [12]:
# Count the recipes
n_wines = len(filtered_data)
print(f"{n_wines} recipes loaded")

150930 recipes loaded


In [13]:
example = filtered_data[25]
print(example)

wine review : New Zealand : Kumeu : Chardonnay : Yields were down in 2015, but intensity is up, giving this medium-bodied, silky wine the potential to drink well through at least 2025. Hickory smoke outlines white peach before ending in a long flurry of lime zest.


## 2. Tokenize the data <a name="tokenize"></a>

In [14]:
# Pad the punctuation, to treat them as separate 'words'
def pad_punctuation(s):
    s = re.sub(f"([{string.punctuation}, '\n'])", r" \1 ", s)
    s = re.sub(" +", " ", s)
    return s


text_data = [pad_punctuation(x) for x in filtered_data]

In [15]:
# Display an example of a recipe
example_data = text_data[25]
example_data

'wine review : New Zealand : Kumeu : Chardonnay : Yields were down in 2015 , but intensity is up , giving this medium - bodied , silky wine the potential to drink well through at least 2025 . Hickory smoke outlines white peach before ending in a long flurry of lime zest . '

In [16]:
# Convert to a Tensorflow Dataset
text_ds = (
    tf.data.Dataset.from_tensor_slices(text_data)
    .batch(BATCH_SIZE)
    .shuffle(1000)
)

In [17]:
# Create a vectorisation layer
vectorize_layer = layers.TextVectorization(
    standardize="lower",
    max_tokens=VOCAB_SIZE,
    output_mode="int",
    output_sequence_length=MAX_LEN + 1,
)

In [18]:
# Adapt the layer to the training set
vectorize_layer.adapt(text_ds)
vocab = vectorize_layer.get_vocabulary()

In [19]:
# Display some token:word mappings
for i, word in enumerate(vocab[:10]):
    print(f"{i}: {word}")

0: 
1: [UNK]
2: :
3: ,
4: .
5: and
6: the
7: wine
8: a
9: of


In [20]:
# Display the same example converted to ints
example_tokenised = vectorize_layer(example_data)
print(example_tokenised.numpy())

[   7   11    2   91  283    2 3527    2   55    2 1943  814  457   18
  773    3   24  300   12  109    3  435   13  162   15  112    3  239
    7    6  601   20   47   75  106   82  501 2671    4 1808  287    1
   65  117  534 1498   18    8  118 6421    9  185  431    4    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0]


## 3. Create the Training Set <a name="create"></a>

In [21]:
# Create the training set of recipes and the same text shifted by one word
def prepare_inputs(text):
    text = tf.expand_dims(text, -1)
    tokenized_sentences = vectorize_layer(text)
    x = tokenized_sentences[:, :-1]
    y = tokenized_sentences[:, 1:]
    return x, y


train_ds = text_ds.map(prepare_inputs)

In [22]:
example_input_output = train_ds.take(1).get_single_element()

In [23]:
# Example Input
example_input_output[0][0]

<tf.Tensor: shape=(80,), dtype=int64, numpy=
array([   7,   11,    2,   21,    2,   25,    2,   65,   28,    2,    1,
         16,   19,   28,    9, 3694,   70,   36,   93,    3, 3932,   70,
        327,    3,  603,   70, 1107,    3, 1806,   70,   41,   93,    3,
       1352,   70,  209, 1143,    5, 1352,   70, 1143,    1,   12,   75,
         15,  138,    5, 2261,    3,  162,   15,  299,    5,  499,   50,
         18,   59,   10,   88,    5,  309,  459,    4, 3226, 2279,    4,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0])>

In [24]:
# Example Output (shifted by one token)
example_input_output[1][0]

<tf.Tensor: shape=(80,), dtype=int64, numpy=
array([  11,    2,   21,    2,   25,    2,   65,   28,    2,    1,   16,
         19,   28,    9, 3694,   70,   36,   93,    3, 3932,   70,  327,
          3,  603,   70, 1107,    3, 1806,   70,   41,   93,    3, 1352,
         70,  209, 1143,    5, 1352,   70, 1143,    1,   12,   75,   15,
        138,    5, 2261,    3,  162,   15,  299,    5,  499,   50,   18,
         59,   10,   88,    5,  309,  459,    4, 3226, 2279,    4,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0])>

## 5. Create the causal attention mask function <a name="causal"></a>

In [25]:
def causal_attention_mask(batch_size, n_dest, n_src, dtype):
    i = tf.range(n_dest)[:, None]
    j = tf.range(n_src)
    m = i >= j - n_src + n_dest
    mask = tf.cast(m, dtype)
    mask = tf.reshape(mask, [1, n_dest, n_src])
    mult = tf.concat(
        [tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)], 0
    )
    return tf.tile(mask, mult)


np.transpose(causal_attention_mask(1, 10, 10, dtype=tf.int32)[0])

array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 0, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 0, 0, 1, 1, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 1, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]], dtype=int32)

## 6. Create a Transformer Block layer <a name="transformer"></a>

In [26]:
class TransformerBlock(layers.Layer):
    def __init__(self, num_heads, key_dim, embed_dim, ff_dim, dropout_rate=0.1):
        super(TransformerBlock, self).__init__()
        self.num_heads = num_heads
        self.key_dim = key_dim
        self.embed_dim = embed_dim
        self.ff_dim = ff_dim
        self.dropout_rate = dropout_rate
        self.attn = layers.MultiHeadAttention(
            num_heads, key_dim, output_shape=embed_dim
        )
        self.dropout_1 = layers.Dropout(self.dropout_rate)
        self.ln_1 = layers.LayerNormalization(epsilon=1e-6)
        self.ffn_1 = layers.Dense(self.ff_dim, activation="relu")
        self.ffn_2 = layers.Dense(self.embed_dim)
        self.dropout_2 = layers.Dropout(self.dropout_rate)
        self.ln_2 = layers.LayerNormalization(epsilon=1e-6)

    def call(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size = input_shape[0]
        seq_len = input_shape[1]
        causal_mask = causal_attention_mask(
            batch_size, seq_len, seq_len, tf.bool
        )
        attention_output, attention_scores = self.attn(
            inputs,
            inputs,
            attention_mask=causal_mask,
            return_attention_scores=True,
        )
        attention_output = self.dropout_1(attention_output)
        out1 = self.ln_1(inputs + attention_output)
        ffn_1 = self.ffn_1(out1)
        ffn_2 = self.ffn_2(ffn_1)
        ffn_output = self.dropout_2(ffn_2)
        return (self.ln_2(out1 + ffn_output), attention_scores)

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "key_dim": self.key_dim,
                "embed_dim": self.embed_dim,
                "num_heads": self.num_heads,
                "ff_dim": self.ff_dim,
                "dropout_rate": self.dropout_rate,
            }
        )
        return config

## 7. Create the Token and Position Embedding <a name="embedder"></a>

In [27]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, max_len, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.max_len = max_len
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
        self.token_emb = layers.Embedding(
            input_dim=vocab_size, output_dim=embed_dim
        )
        self.pos_emb = layers.Embedding(input_dim=max_len, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "max_len": self.max_len,
                "vocab_size": self.vocab_size,
                "embed_dim": self.embed_dim,
            }
        )
        return config

## 8. Build the Transformer model <a name="transformer_decoder"></a>

In [28]:
inputs = layers.Input(shape=(None,), dtype=tf.int32)
x = TokenAndPositionEmbedding(MAX_LEN, VOCAB_SIZE, EMBEDDING_DIM)(inputs)
x, attention_scores = TransformerBlock(
    N_HEADS, KEY_DIM, EMBEDDING_DIM, FEED_FORWARD_DIM
)(x)
outputs = layers.Dense(VOCAB_SIZE, activation="softmax")(x)
gpt = models.Model(inputs=inputs, outputs=[outputs, attention_scores])
gpt.compile("adam", loss=[losses.SparseCategoricalCrossentropy(), None])

In [29]:
gpt.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, None)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ token_and_position_embedding    │ (None, None, 256)      │     2,580,480 │
│ (TokenAndPositionEmbedding)     │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block               │ [(None, None, 256),    │       658,688 │
│ (TransformerBlock)              │ (None, 2, None, None)] │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, None, 10000)    │     2,570,000 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,809,168 (22.16 MB)

 Trainable params: 5,809,168 (22.16 MB)

 Non-trainable params: 0 (0.00 B)

In [33]:
if LOAD_MODEL:
    # model.load_weights('./models/model')
    gpt = models.load_model("./models/gpt", compile=True)

## 9. Train the Transformer <a name="train"></a>

In [34]:
# Create a TextGenerator checkpoint
class TextGenerator(callbacks.Callback):
    def __init__(self, index_to_word, top_k=10):
        self.index_to_word = index_to_word
        self.word_to_index = {
            word: index for index, word in enumerate(index_to_word)
        }

    def sample_from(self, probs, temperature):
        probs = probs ** (1 / temperature)
        probs = probs / np.sum(probs)
        return np.random.choice(len(probs), p=probs), probs

    def generate(self, start_prompt, max_tokens, temperature):
        start_tokens = [
            self.word_to_index.get(x, 1) for x in start_prompt.split()
        ]
        sample_token = None
        info = []
        while len(start_tokens) < max_tokens and sample_token != 0:
            x = np.array([start_tokens])
            y, att = self.model.predict(x, verbose=0)
            sample_token, probs = self.sample_from(y[0][-1], temperature)
            info.append(
                {
                    "prompt": start_prompt,
                    "word_probs": probs,
                    "atts": att[0, :, -1, :],
                }
            )
            start_tokens.append(sample_token)
            start_prompt = start_prompt + " " + self.index_to_word[sample_token]
        print(f"\ngenerated text:\n{start_prompt}\n")
        return info

    def on_epoch_end(self, epoch, logs=None):
        self.generate("wine review", max_tokens=80, temperature=1.0)

In [36]:
# Create a model save checkpoint
model_checkpoint_callback = callbacks.ModelCheckpoint(
    filepath="./checkpoint/checkpoint.weights.h5",
    save_weights_only=True,
    save_freq="epoch",
    verbose=0,
)

tensorboard_callback = callbacks.TensorBoard(log_dir="./logs")

# Tokenize starting prompt
text_generator = TextGenerator(vocab)

In [37]:
gpt.fit(
    train_ds,
    epochs=EPOCHS,
    callbacks=[model_checkpoint_callback, tensorboard_callback, text_generator],
)

Epoch 1/5
4716/4717 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 2.5346
generated text:
wine review : australia : south australia : shiraz : an unoaked mouthful of wine connection with [UNK] , raspberry and cherry fruit and moderate acid , prickly up a bit of tannin , yields a leathery finish . it ' s medium - weight is solid and dry . 

4717/4717 ━━━━━━━━━━━━━━━━━━━━ 181s 37ms/step - loss: 2.5345
Epoch 2/5
4716/4717 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 1.9637
generated text:
wine review : us : california : merlot : some good cherry and blackberry fruit . feels youthful in the mouth , this wine has a firm foundation of tannins , acids , new oak and style . try with an ager . 

4717/4717 ━━━━━━━━━━━━━━━━━━━━ 144s 31ms/step - loss: 1.9637
Epoch 3/5
4716/4717 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 1.8773
generated text:
wine review : france : southwest france : white blend : a is a blend of [UNK] , pinot blanc , based on the chardonnay - chardonnay - based category . this powerful , 

In [40]:
import os

# Create the directory if it doesn't exist
os.makedirs("./models", exist_ok=True)

# Save the final model
gpt.save("./models/gpt.keras")

# 3. Generate text using the Transformer

In [41]:
def print_probs(info, vocab, top_k=5):
    for i in info:
        highlighted_text = []
        for word, att_score in zip(
            i["prompt"].split(), np.mean(i["atts"], axis=0)
        ):
            highlighted_text.append(
                '<span style="background-color:rgba(135,206,250,'
                + str(att_score / max(np.mean(i["atts"], axis=0)))
                + ');">'
                + word
                + "</span>"
            )
        highlighted_text = " ".join(highlighted_text)
        display(HTML(highlighted_text))

        word_probs = i["word_probs"]
        p_sorted = np.sort(word_probs)[::-1][:top_k]
        i_sorted = np.argsort(word_probs)[::-1][:top_k]
        for p, i in zip(p_sorted, i_sorted):
            print(f"{vocab[i]}:   \t{np.round(100*p,2)}%")
        print("--------\n")

In [47]:
info = text_generator.generate(
    "wine review : us", max_tokens=20, temperature=1.0
)


generated text:
wine review : us : california : chardonnay : like anderson and a fine acidity saves it with being a



In [48]:
print_probs(info, vocab)

::   	100.0%
-:   	0.0%
grosso:   	0.0%
port:   	0.0%
new:   	0.0%
--------



california:   	66.43000030517578%
washington:   	18.170000076293945%
new:   	6.46999979019165%
oregon:   	6.170000076293945%
virginia:   	1.090000033378601%
--------



::   	99.98999786376953%
-:   	0.0%
valley:   	0.0%
other:   	0.0%
hills:   	0.0%
--------



cabernet:   	23.799999237060547%
chardonnay:   	10.829999923706055%
pinot:   	9.720000267028809%
merlot:   	7.369999885559082%
sauvignon:   	6.71999979019165%
--------



::   	99.83000183105469%
-:   	0.14000000059604645%
blanc:   	0.009999999776482582%
blend:   	0.009999999776482582%
grosso:   	0.0%
--------



...

,:   	68.30999755859375%
and:   	5.739999771118164%
that:   	4.679999828338623%
on:   	2.7799999713897705%
make:   	1.909999966621399%
--------



it:   	58.7599983215332%
the:   	23.950000762939453%
this:   	16.09000015258789%
you:   	0.10000000149011612%
its:   	0.10000000149011612%
--------



from:   	38.86000061035156%
.:   	15.3100004196167%
,:   	8.720000267028809%
for:   	8.039999961853027%
with:   	6.320000171661377%
--------



being:   	21.600000381469727%
this:   	14.25%
the:   	11.65999984741211%
a:   	5.900000095367432%
from:   	4.199999809265137%
--------



too:   	30.510000228881836%
a:   	8.520000457763672%
heavy:   	4.389999866485596%
dry:   	3.440000057220459%
released:   	2.680000066757202%
--------



In [51]:
info = text_generator.generate(
    "wine review : italy", max_tokens=80, temperature=5
)


generated text:
wine review : italy assyrtiko nicest savigny favorably malbec large once needles noted cigarette after balsamic lips shoulder vitamin befits natural prosecco territory perceived vibrancy similar iconic 4–5 chilly haut leading sierra loaded vinho sinuous 2011 petal bill carpet shoulder golden melded trying renaissance thirst sweat spicy raspingly zonin helm play wrong no excited save contributed grow exceptionally simonsberg utilized heat crab marco russell masculine grounded capture department one tone russell oaky copious weight deeper distinguished dessert complications imports carabella



In [50]:
print_probs(info, vocab)

::   	1.1399999856948853%
other:   	0.10999999940395355%
grosso:   	0.10999999940395355%
france:   	0.10000000149011612%
[UNK]:   	0.09000000357627869%
--------



::   	0.7599999904632568%
grosso:   	0.15000000596046448%
veneto:   	0.09000000357627869%
sicily:   	0.09000000357627869%
blend:   	0.07999999821186066%
--------



::   	1.0499999523162842%
grosso:   	0.17000000178813934%
other:   	0.10999999940395355%
[UNK]:   	0.07999999821186066%
valley:   	0.07000000029802322%
--------



::   	0.47999998927116394%
grosso:   	0.10999999940395355%
red:   	0.10999999940395355%
sauvignon:   	0.10999999940395355%
white:   	0.10000000149011612%
--------



::   	0.4300000071525574%
sauvignon:   	0.14000000059604645%
blanc:   	0.14000000059604645%
weissburgunder:   	0.11999999731779099%
-:   	0.11999999731779099%
--------



...

.:   	0.20999999344348907%
on:   	0.18000000715255737%
[UNK]:   	0.10000000149011612%
displayed:   	0.10000000149011612%
with:   	0.09000000357627869%
--------



.:   	0.17000000178813934%
with:   	0.12999999523162842%
on:   	0.11999999731779099%
[UNK]:   	0.11999999731779099%
,:   	0.10999999940395355%
--------



.:   	0.23000000417232513%
with:   	0.14000000059604645%
on:   	0.12999999523162842%
,:   	0.10000000149011612%
through:   	0.10000000149011612%
--------



.:   	0.17000000178813934%
,:   	0.10999999940395355%
on:   	0.10999999940395355%
than:   	0.10000000149011612%
and:   	0.10000000149011612%
--------



of:   	0.14000000059604645%
.:   	0.12999999523162842%
out:   	0.10000000149011612%
the:   	0.10000000149011612%
your:   	0.10000000149011612%
--------



In [52]:
info = text_generator.generate(
    "wine review : germany", max_tokens=15, temperature=0.5
)
print_probs(info, vocab)


generated text:
wine review : germany : mosel - saar - ruwer : riesling : this is



::   	100.0%
-:   	0.0%
grosso:   	0.0%
giallo:   	0.0%
weissburgunder:   	0.0%
--------



mosel:   	87.01000213623047%
rheingau:   	5.03000020980835%
rheinhessen:   	4.130000114440918%
pfalz:   	3.4600000381469727%
nahe:   	0.1899999976158142%
--------



::   	55.470001220703125%
-:   	44.529998779296875%
blanc:   	0.0%
grosso:   	0.0%
weissburgunder:   	0.0%
--------



saar:   	100.0%
riesling:   	0.0%
ruwer:   	0.0%
mosel:   	0.0%
donauland:   	0.0%
--------



...

::   	100.0%
-:   	0.0%
blend:   	0.0%
grosso:   	0.0%
port:   	0.0%
--------



riesling:   	100.0%
sauvignon:   	0.0%
kerner:   	0.0%
chardonnay:   	0.0%
sparkling:   	0.0%
--------



::   	100.0%
-:   	0.0%
blend:   	0.0%
grosso:   	0.0%
,:   	0.0%
--------



this:   	81.91000366210938%
a:   	11.699999809265137%
[UNK]:   	1.7599999904632568%
the:   	1.3899999856948853%
from:   	0.6899999976158142%
--------



is:   	37.5099983215332%
wine:   	29.229999542236328%
medium:   	25.520000457763672%
auslese:   	1.3300000429153442%
[UNK]:   	1.100000023841858%
--------

